In [2]:
from sklearn.datasets import fetch_20newsgroups
import keras
from keras import models
from keras import layers
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Flatten, TextVectorization, InputLayer, Input, Dropout
from keras.layers import Embedding
from keras.utils import to_categorical
import tensorflow as tf
import numpy as np
from nltk.stem.lancaster import LancasterStemmer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
tf.config.run_functions_eagerly(True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# all = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)
train = fetch_20newsgroups(subset='train', shuffle=True, random_state = 42)
test = fetch_20newsgroups(subset='test', shuffle=True, random_state = 42)
unique, counts = np.unique(train.target, return_counts=True)
print(counts)
# print(train.data[:10])
# print(train.target[:10])

[480 584 591 590 578 593 585 594 598 597 600 595 591 594 593 599 546 564
 465 377]


In [27]:
MAX_SEQUENCE_LENGTH = 1000
MAX_WORDS = 20000

def remove_stop_words_and_stem(text):
    text = word_tokenize(text.lower())
    stemmer = LancasterStemmer()
    stop_words = set(stopwords.words('english'))
    text = [stemmer.stem(word.lower()) for word in text]
    text = [word for word in text if word.isalpha() and not word in stop_words]
    return ' '.join(text)

# train.data = list(map(remove_stop_words_and_stem, train.data))
# test.data = list(map(remove_stop_words_and_stem, test.data))

# X_train = list(map(remove_stop_words_and_stem, X_train))
# X_test = list(map(remove_stop_words_and_stem, X_test))

# vectorize_layer = TextVectorization(
#  max_tokens=MAX_NB_WORDS,
#  output_mode='int',
#  output_sequence_length=MAX_SEQUENCE_LENGTH)
# vectorize_layer.adapt(train.data)

tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(train.data)
sequences = tokenizer.texts_to_sequences(train.data)

word_index = tokenizer.word_index

print('Found %s unique tokens.' % len(word_index))

train_data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
train_target = to_categorical(np.asarray(train.target))
print('Shape of data tensor:', train_data.shape)
# print('Shape of label tensor:', train_target.shape)

sequences1 = tokenizer.texts_to_sequences(test.data)
test_data = pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)
test_target = to_categorical(np.asarray(test.target))


Found 134142 unique tokens.
Shape of data tensor: (11314, 1000)


In [28]:
# here is a rule of thumb that says min(50, num_categories/2)
# If we’re in a hurry, one rule of thumb is to use the fourth root of the
# total number of unique categorical elements while another is that the embedding
# dimension should be approximately 1.6 times the square root of the number of
# unique elements in the category, and no less than 600.
EMBEDDING_DIM = 300

# decay_steps = 1000
# initial_learning_rate = 0.1
# lr_scheduler = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate, decay_steps)

optimizer = tf.keras.optimizers.Adam(
    # learning_rate=0.0008,
    # weight_decay=0.0001,
)
# optimizer = tf.keras.optimizers.SGD()
network = models.Sequential()
network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
network.add(Flatten())
network.add(layers.Dense(256, activation='relu'))
network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
network.compile(optimizer=optimizer,#<--'SGD'
                loss='categorical_crossentropy',
                metrics=['accuracy'])


In [ ]:
history = network.fit(train_data,
                    train_target,
                    epochs=100,
                    batch_size=32,
                    validation_data=(test_data, test_target))

Epoch 1/100
354/354 [==============================] - 28s 79ms/step - loss: 2.5620 - accuracy: 0.2813 - val_loss: 1.3639 - val_accuracy: 0.5774
Epoch 2/100
354/354 [==============================] - 28s 78ms/step - loss: 0.3658 - accuracy: 0.9000 - val_loss: 1.0012 - val_accuracy: 0.7053
Epoch 3/100
133/354 [==========>...................] - ETA: 14s - loss: 0.0214 - accuracy: 0.9967

In [ ]:
# loss: 0.0026 - accuracy: 0.9994 - val_loss: 1.2574 - val_accuracy: 0.7442
# MAX_SEQUENCE_LENGTH = 1000
# MAX_WORDS = 20000
# EMBEDDING_DIM = 300
# optimizer = tf.keras.optimizers.Adam()
# network = models.Sequential()
# network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
# network.add(Flatten())
# network.add(layers.Dense(256, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
# network.compile(optimizer=optimizer,#<--'SGD'
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy'])
# history = network.fit(train_data,
#                     train.target,
#                     epochs=200,
#                     batch_size=64,
#                     validation_data=(test_data, test.target))



# oss: 0.0040 - accuracy: 0.9994 - val_loss: 0.9964 - val_accuracy: 0.7540
# categorical
# embed 50, max words 1000, max seq 300
# optimizer = tf.keras.optimizers.Adam()
# network = models.Sequential()
# network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
# network.add(Flatten())
# network.add(layers.Dense(256, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
# network.compile(optimizer=optimizer,#<--'SGD'
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])



# loss: 0.0038 - accuracy: 0.9992 - val_loss: 1.4800 - val_accuracy: 0.7002
#  jak wyżej
# network.add(layers.Dense(512, activation='relu'))
# network.add(layers.Dense(256, activation='relu'))
# network.add(layers.Dense(128, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classe

# jak wyżej
# emebedding 60 = 1.6 * sqrt
# loss: 1.2944e-05 - accuracy: 1.0000 - val_loss: 1.2793 - val_accuracy: 0.7102
# network = models.Sequential()
# network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
# network.add(Flatten())
# network.add(layers.Dense(512, activation='relu'))
# network.add(layers.Dense(128, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
# network.compile(optimizer=optimizer,#<--'SGD'
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])


# loss: 1.6622e-04 - accuracy: 0.9999 - val_loss: 1.7396 - val_accuracy: 0.6747
# network = models.Sequential()
# network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
# network.add(Flatten())
# network.add(layers.Dense(800, activation='relu'))
# network.add(layers.Dense(64, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
# network.compile(optimizer=optimizer,#<--'SGD'
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])

#  loss: 4.0713e-08 - accuracy: 1.0000 - val_loss: 3.0106 - val_accuracy: 0.6508
# network = models.Sequential()
# network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
# network.add(Flatten())
# network.add(layers.Dense(128, activation='relu'))
# network.add(layers.Dense(64, activation='relu'))
# network.add(layers.Dense(32, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
# network.compile(optimizer=optimizer,#<--'SGD'
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])


# loss: 7.6941e-07 - accuracy: 1.0000 - val_loss: 2.0702 - val_accuracy: 0.6827
# network = models.Sequential()
# network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
# network.add(Flatten())
# network.add(layers.Dense(256, activation='relu'))
# network.add(layers.Dense(32, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
# network.compile(optimizer=optimizer,#<--'SGD'
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])


# loss: 1.3598e-07 - accuracy: 1.0000 - val_loss: 1.3201 - val_accuracy: 0.7228
# network = models.Sequential()
# network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
# network.add(Flatten())
# network.add(layers.Dense(256, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
# network.compile(optimizer=optimizer,#<--'SGD'
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])


#  loss: 2.0386e-07 - accuracy: 1.0000 - val_loss: 1.3548 - val_accuracy: 0.6920
# MAX_SEQUENCE_LENGTH = 500
# MAX_WORDS = 10000

# loss: 4.4888e-06 - accuracy: 1.0000 - val_loss: 1.1738 - val_accuracy: 0.7095
# network = models.Sequential()
# network.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
# network.add(Flatten())
# network.add(layers.Dense(513, activation='relu'))
# network.add(layers.Dense(20, activation='softmax')) # 20 because 20 classes
# network.compile(optimizer=optimizer,#<--'SGD'
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])


# z 256 neuronów w jednej wartstwie
# loss: 7.9624e-08 - accuracy: 1.0000 - val_loss: 1.3645 - val_accuracy: 0.7189 <--- 1. epoka, później spadało
# history = network.fit(train_data,
#                     train_target,
#                     epochs=100,
#                     batch_size=16,
#                     validation_data=(test_data, test_target))


#  loss: 2.5353e-07 - accuracy: 1.0000 - val_loss: 1.2517 - val_accuracy: 0.7212
# history = network.fit(train_data,
#                     train_target,
#                     epochs=100,
#                     batch_size=32,
#                     validation_data=(test_data, test_target))


# loss: 1.3223e-07 - accuracy: 1.0000 - val_loss: 1.2755 - val_accuracy: 0.7209 <-- druga epoka, później spada
# history = network.fit(train_data,
#                     train_target,
#                     epochs=100,
#                     batch_size=128,
#                     validation_data=(test_data, test_target))

In [ ]:
history_dict = history.history
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

ep=len(acc)
epochs = range(1, ep)

# "bo" is for "blue dot"
plt.plot(epochs,acc[1:ep], 'bo', label='Training accuracy')
# b is for "solid blue line"
plt.plot(epochs, val_acc[1:ep], 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()